In [1]:
import tushare as ts
import pandas as pd
from influxdb import InfluxDBClient, DataFrameClient
from datetime import datetime, timedelta
from tqdm import tqdm
import os

In [2]:
ts.set_token('53cd3b985c649c978160c6ec04bce24f4fbd2ebcb4673e8f2fba9a43')

pro = ts.pro_api()

In [3]:
now = datetime.utcnow() + timedelta(hours=8) - timedelta(days=2)
yesterday = now.strftime("%Y%m%d")

In [4]:
yesterday

'20200513'

In [5]:
df = pro.daily(trade_date=yesterday)

In [6]:
df['trade_date'] = pd.to_datetime(df['trade_date'])

In [7]:
df.head()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,000838.SZ,2020-05-13,2.65,2.67,2.63,2.64,2.65,-0.01,-0.3774,27163.88,7182.272
1,000839.SZ,2020-05-13,3.21,3.22,3.17,3.20,3.23,-0.03,-0.9288,162930.55,52048.053
2,000848.SZ,2020-05-13,6.26,6.58,6.18,6.48,6.28,0.20,3.1847,284099.96,182068.111
3,000850.SZ,2020-05-13,3.62,3.65,3.59,3.64,3.64,0.00,0.0000,48064.04,17391.123
4,000851.SZ,2020-05-13,7.70,7.71,7.26,7.64,8.06,-0.42,-5.2109,1022770.35,769409.782


In [10]:
client = InfluxDBClient('localhost', 8086, database='stock')

In [11]:
js = df.to_dict('records')

In [12]:
js[0]

{'ts_code': '000838.SZ',
 'trade_date': Timestamp('2020-05-13 00:00:00'),
 'open': 2.65,
 'high': 2.67,
 'low': 2.63,
 'close': 2.64,
 'pre_close': 2.65,
 'change': -0.01,
 'pct_chg': -0.3774,
 'vol': 27163.88,
 'amount': 7182.272}

In [13]:
processed_js = []
for record in js:
    time = record.pop('trade_date')
    companyId = record.pop('ts_code')
    processed_js.append({
        "measurement": "stock",
        "tags": {
            'companyId': companyId
        },
        "time": time,
        "fields": record
    })

In [14]:
processed_js[0]

{'measurement': 'stock',
 'tags': {'companyId': '000838.SZ'},
 'time': Timestamp('2020-05-13 00:00:00'),
 'fields': {'open': 2.65,
  'high': 2.67,
  'low': 2.63,
  'close': 2.64,
  'pre_close': 2.65,
  'change': -0.01,
  'pct_chg': -0.3774,
  'vol': 27163.88,
  'amount': 7182.272}}

In [15]:
client.write_points(processed_js)

True

In [4]:
df_client = DataFrameClient('116.62.23.105', 8086, database='stock', username='influx', password='2020liufeng')

In [8]:
df_client.query("select * from stock where companyId='000001.SZ'")['stock']

,amount,change,close,companyId,high,low,open,pct_chg,pre_close,vol
2020-05-14 00:00:00+00:00,1259440.848,-0.33,13.3,000001.SZ,13.59,13.22,13.55,-2.4211,13.63,944672.09
